# HomeWork 3


Домашнее задание: максимум 20 баллов

1. Побейте метрику на лидерборде map@10 = 0.063 для userKnn модели с семинара (5 баллов)

2. Реализуйте эксперименты с кастомной моделю kNN с семинара. Результат - ноутбук(и) (максимум 15 баллов)

(Вы можете отрефакторить код из userknn.py по желанию или не трогать его)

Что можно сделать в ноутбуке:

- придумать, что делать с холодными пользователями в тесте. 
- Сделайте рекомендации для них (обратите внимание на rectools.models.popular) (3 балла)
  - UserPopular
  - UserPopularTopCategory
  - NewItems
- Сделать кол-во рекомендаций равным N, а не меньше N (3 балла)
  - Popular
  - NewItems
  - ?LastTopViewed
- Реализовать тюнинг гиперпараметров (например, векторного расстояния или типов kNN моделей (implicit/rectools/...)) и сделать выводы (3 балла)
  - реализовать другие варианты ранжированивания айтемов похожих пользователей и сделать выводы (3 балла)
  - провести эксперименты с параметрами оффлайн валидации и сделать выводы (3 балла)
    - Optuna
    - Wandb, ClearML, MetaFlow

1. Оберните модель в сервис (максимум 12 баллов)

    предпочтительный онлайн вариант: 
    - обучаете модель в ноутбуке, 
    - сохраняете обученную модель (pickle, dill), 
    - при запуске сервиса ее поднимаете и запрашиваете рекомендации "на лету" (12 баллов)
    или оффлайн вариант: 
    - предварительно посчитайте рекомендации для всех пользователей
    - сохраните и запрашивайте их (6 баллов)

Хороший pull request - это:  
    - наличие описания (в идеале что сделано - по пунктам)  
    - код по стандарту PEP8  
    - легкая читаемость и воспроизводимость кода  
    - комментарии и объяснения. В ipynb пользуйтесь силой маркдауна. В скриптах пишите комментарии и докстринг.  
    - обоснование схемы валидации  
    - анализ метрики качества  


In [1]:
import pandas as pd
import numpy as np

#!pip install rectools
#!pip install seaborn
#!pip install pyarrow
# !pip install Jinja2

In [2]:
inter = pd.read_csv(
    "../data/interactions.csv",
)

items = pd.read_csv(
    '../data/items.csv',
)

# users = pd.read_csv(
#     '../data/users.csv',
# )

In [3]:
inter.head(3)

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0


In [4]:
inter = inter.rename(columns={"last_watch_dt": "datetime"})

inter["datetime"] = pd.to_datetime(inter["datetime"])

inter["weight"] = inter["total_dur"] * inter["watched_pct"]
# inter['weight'] = np.where(inter['watched_pct'] > 25, 1, 0) * inter['total_dur']

# Задание 1

In [5]:
from rectools.metrics import (
    Precision,
    Recall,
    NDCG,
    MAP,
    Serendipity,
    MeanInvUserFreq,
    calc_metrics,
)

from implicit.nearest_neighbours import TFIDFRecommender
from rectools.models import RandomModel, PopularModel, PopularInCategoryModel, ImplicitItemKNNWrapperModel
from rectools import Columns
from rectools.model_selection.time_split import TimeRangeSplitter
from rectools.dataset import Interactions, Dataset

/home/kartushov/pet_projects/itmo/itmo.recsys/RecoServiceTemplate/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/kartushov/pet_projects/itmo/itmo.recsys/RecoServiceTemplate/.venv/lib/python3.10/site-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


In [6]:
K = 10

n_splits = 3

splitter = TimeRangeSplitter(
    test_size="14D",
    n_splits=n_splits,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=True,
)

models = {
    "TDIDF": TFIDFRecommender(K=K),
}

metrics = {
    "MAPTop-1": MAP(k=1),
    "MAPTop-5": MAP(k=5),
    "MAPTop-10": MAP(k=10),
    "NDCG@1": NDCG(k=1),
    "NDCG@5": NDCG(k=5),
    "NDCG@10": NDCG(k=10),
    "Precision@1": Precision(k=1),
    "Precision@5": Precision(k=5),
    "Precision@10": Precision(k=10),
    "Recall@1": Recall(k=1),
    "Recall@5": Recall(k=5),
    "Recall@10": Recall(k=10),
    "Novelty@1": MeanInvUserFreq(k=1),
    "Novelty@5": MeanInvUserFreq(k=5),
    "Novelty@10": MeanInvUserFreq(k=10),
    "Serendipity": Serendipity(k=1),
    "Serendipity": Serendipity(k=5),
    "Serendipity": Serendipity(k=10),
}


In [7]:
import time
import pickle
from tqdm import tqdm


class RecSysModule:
    def __init__(
        self, model_dict: dict = None, metrics_dict: dict = None, splitter: TimeRangeSplitter = None, k: int = 10
    ):
        """
        Initialize the RecSysModule object.

        Parameters
        ----------
        model_dict : dict, optional
            Dictionary containing models, by default None
        metrics_dict : dict, optional
            Dictionary containing metrics, by default None
        splitter : TimeRangeSplitter, optional
            TimeRangeSplitter object for splitting data, by default None
        k : int, optional
            Value of 'k' for top-k recommendations, by default 10

        Raises
        ------
        ValueError
            If splitter is not provided.
        ValueError
            If metrics_dict is not provided.
        ValueError
            If model_dict is not provided.
        ValueError
            If no metrics are provided.
        ValueError
            If no models are provided.
        ValueError
            If k is negative.
        """
        if splitter is None:
            raise ValueError("Please provide splitter")
        if metrics_dict is None:
            raise ValueError("Please provide metrics_dict")
        if model_dict is None:
            raise ValueError("Please provide model_dict")
        if len(metrics_dict) == 0:
            raise ValueError("Please provide at least one metric")
        if len(model_dict) == 0:
            raise ValueError("Please provide at least one model")
        if k < 0:
            raise ValueError("k cannot be negative")

        self.model_dict = model_dict
        self.metrics_dict = metrics_dict
        self.splitter = splitter
        self.k = k


    def fold_iterator(self, dataframe: pd.DataFrame, treshold=5):        
        count_views_items = dataframe.groupby('user_id')['item_id'].count()
        train_ids = list(count_views_items[count_views_items > treshold].index)
        test_ids = list(count_views_items[count_views_items <= treshold].index)

        test_ids = [user_id for user_id in test_ids if user_id in train_ids]

        return train_ids, test_ids


    def evaluate(self, interaction: pd.DataFrame) -> pd.DataFrame:
        results = []
        
        fold_iterator = splitter.split(interaction, collect_fold_stats=True) 
        for i_fold, (train_ids, test_ids, fold_info) in enumerate(fold_iterator):
            print(f"\n==================== Fold {i_fold}")
            train = interaction.df.iloc[train_ids].copy()
            test = interaction.df.iloc[test_ids][Columns.UserItem].copy()
            catalog = train[Columns.Item].unique()

            start_time = time.time()

            test_users = np.unique(test[Columns.User])

            for model_name, model in models.items():
                userknn_model = UserKnn(model=model, N_users=50)
                userknn_model.fit(train)
                model.fit(dataset)
                recos = userknn_model.predict(test)   
                with open(f'../models/{model_name}_{i_fold}.pickle', 'wb') as f:
                    pickle.dump(model, f)

                recos = model.recommend(
                    users=test_users,
                    dataset=dataset,
                    k=self.k,
                    filter_viewed=True,
                )

                metric_values = calc_metrics(
                        metrics,
                        reco=recos,
                        interactions=test,
                        prev_interactions=train,
                        catalog=catalog,
                    )
                end_time = time.time()
                evaluation_time = end_time - start_time

                res = {"model": model_name, "evaluation_time": evaluation_time}
                res.update(metric_values)

        self.metrics_result = results


    def visualize_results(self):
            pivot_results = pd.DataFrame(self.metrics_result).drop(columns="fold").groupby(["model"], sort=False).agg(["mean", "std"])
            mean_metric_subset = [(metric, agg) for metric, agg in pivot_results.columns if agg == 'mean']
            return (
                pivot_results.style
                .highlight_min(subset=mean_metric_subset, color='lightcoral', axis=0)
                .highlight_max(subset=mean_metric_subset, color='lightgreen', axis=0)
            )


    def visualize_analysis(self, model, dataset, user_ids, item_data):
        recos = model.predict(item_data)
        recos = recos[recos[Columns.User].isin(user_ids)]
        recos = recos.merge(item_data, on=[Columns.Item], how='left')[['title', 'genres', Columns.User, Columns.Item]]
        
        history = dataset.interactions.df
        history = history[history[Columns.User].isin(user_ids)]
        history = history.merge(item_data, on=[Columns.Item], how='left')[['title', 'genres', Columns.User, Columns.Item]]

        for user in user_ids:
            print(f"User {user} history:")
            history_items = history[Columns.Item].unique()
            print(f"Total count views user: {history[history[Columns.User] == user][Columns.Item].nunique()}")
            for item in history_items:
                _his_item_dataset = dataset.interactions.df
                item_title = history[history[Columns.Item] == item]['title'].values[0]
                # unique equal count cuz 1 user ~ 1 item in this set
                item_unviews = _his_item_dataset[_his_item_dataset[Columns.Item] == item][Columns.User].nunique()
                print(f"{item_title} | total views: {item_unviews}")
            display(history[history[Columns.User] == user])
            print()
            print(f"User {user} recommendations:")
            display(recos[recos[Columns.User] == user])
            print()
            print()

In [8]:
interactions = Interactions(inter)

recsys = RecSysModule(
    model_dict=models,
    metrics_dict=metrics,
    splitter=splitter,
    k=10,
)

# FIT/PREDICT

In [23]:
interactions.df.head()

,user_id,item_id,datetime,total_dur,watched_pct,weight
0,176549,9506,2021-05-11,4250,72.0,306000.0
1,699317,1659,2021-05-29,8317,100.0,831700.0
2,656683,7107,2021-05-09,10,0.0,0.0
3,864613,7638,2021-07-05,14483,100.0,1448300.0
4,964868,9506,2021-04-30,6725,100.0,672500.0


In [25]:
tfidf_model = TFIDFRecommender(K=10)

In [26]:
df_train = interactions.df
dataset = Dataset.construct(
    interactions_df=df_train,
    user_features_df=None,
    item_features_df=None,
)
catalog = df_train[Columns.Item].unique()

In [27]:
tfidf_model.fit(df_train) 

  1%|          | 7401/962179 [02:21<5:03:11, 52.49it/s]


In [10]:
with open(f'../models/TFIDF.pickle', 'wb') as f:
    pickle.load(tfidf_model, f)

NameError: name 'tfidf_model' is not defined

In [29]:
tfidf_recos = userknn.predict(df_train)

AttributeError: 'TFIDFRecommender' object has no attribute 'predict'

In [ ]:
tfidf_recos.to_parquet('data/tfidf.parquet')

NameError: name 'tfidf_recos' is not defined

In [ ]:
tfidf_recos.head()